In [114]:
import pandas as pd
import numpy as np
import polars as pl
import plotly.express as px
from pathlib import Path

data_path = Path("data")


def load_data(ps_codes,chemicals_to_check):
	return pl.read_parquet(
		data_path.joinpath("gsa_swrcb_wq.parquet")
		).filter(
			# pl.col('PS Code').str.strip() == f'{id}'
			pl.col('PS Code').str.strip().is_in(ps_codes)
			).select(
				pl.col(pl.Utf8).str.strip()
				).with_columns(
					pl.col("Sample Date").str.strptime(pl.Date, fmt="%m-%d-%Y").alias("Date")
					).filter(
						pl.col('Date') > pl.date(2015,1,1)
						# ).filter(
						# 	pl.col('Date') < pl.date(2022,1,1)
							).filter(
								pl.col('Analyte Name').is_in(chemicals_to_check)
								# ).with_columns(
								# 	pl.col('Result').cast(pl.Float32,strict=True).fill_null(-1).alias("reading")
									).sort("Date", descending=True)




class GSA:
	def __init__(self, dfs,name):
		self.name = name
		self.public_wells = dfs['public_wells'].pipe(lambda df:df.loc[df['GSA'] == name])
		self.ps_codes = [i for i in self.public_wells['PS Code'].unique()]
		# st.dataframe(dfs['chemical_crosswalk'])
		self.chemical_names = dfs['chemical_crosswalk'].pipe(lambda df:df.loc[df['GSA'] == name])
		self.chemicals_to_check = [i for i in self.chemical_names['public_name']]
		self.chemical_titles = [i for i in self.chemical_names['pp_title']]


		self.df = load_data(self.ps_codes,self.chemicals_to_check).to_pandas()
		self.df = self.df.astype({'Sample Date': 'datetime64[ns]', 'Result': 'float64',"MCL":'float64'},errors='ignore')

class Analyte:
	def __init__(self, gsa,ps_code,title):
		self.title = title
		self.chemical_row = gsa.chemical_names.loc[gsa.chemical_names['pp_title'] == title].iloc[0]
		self.public_name = self.chemical_row['public_name']
		self.pp_units = self.chemical_row['units']

		self.ps_code = ps_code
		self.gsa = gsa

		self.data = gsa.df.loc[(gsa.df['PS Code'] == ps_code) & (gsa.df['Analyte Name'] == self.public_name)].reset_index(drop=True)
		self.data['title'] = self.title
		self.data.sort_values(by='Date',ascending=True)

		if self.data.shape[0] == 0:
			# print(f"**Warning: No data for {self.title}**")
			self.all_df = pd.DataFrame(
				{'title': [self.title],}
			)
			# return
			self.is_empty = True
			self.minimum_threshold = "NA"
			self.minimum_threshold_calculated = False
		
		else:
			self.is_empty = False
				
			
			self.units = [i for i in self.data['Units of Measure'].unique()]
			if len(self.units) == 1:
				self.units = self.units[0]
				if self.units == self.pp_units:
					pass
				else:
					if self.units == 'UG/L' and self.pp_units == 'MG/L':
						self.data['Result'] = self.data['Result'].apply(lambda x: x / 1000)
						self.data['MCL'] = self.data['MCL'].apply(lambda x: x / 1000)
						self.data['Units of measure'] = self.pp_units
					elif self.units == 'MG/L' and self.pp_units == 'UG/L':
						self.data['Result'] = self.data['Result'].apply(lambda x: x * 1000)
						self.data['MCL'] = self.data['MCL'].apply(lambda x: x * 1000)
						self.data['Units of measure'] = self.pp_units

					else:
						print(f"**Warning: Units don't match for {self.title}**")
						print(f"Public: {self.pp_units}")
						print(f"SWRCB: {self.units}")

				


					# else:
			format_flt = lambda s: s if isinstance(s, str) else f"{float(s):.2f}"
						
			self.all_df = self.data[["title","Date",'Result',"Units of Measure","MCL"]].sort_values(
				by='Date',ascending=False
				).replace(
					np.nan, 'ND', regex=True
						)

			self.minimum_threshold = self.get_mt()

	# def convert_units(self,units):


	def get_mt(self):
		data = self.data
		MCLs = [i for i in data['MCL'].unique() if i is not None]
		# print(f"MCL = {MCLs}")
		# print(f"MCL = {float(MCLs[1])}")
		if len(MCLs) == 1:
			mcl = float(MCLs[0])
		else:
			mcl = float(max([float(x) for x in MCLs]))

		try:
			# find max level for 2015-2021
			filter_data = data.loc[(data['Date'] > '2015-01-01') & (data['Date'] < '2022-01-01')].reset_index(drop=True)
			# st.dataframe(filter_data)

			max_level = filter_data['Result'].max()
			max_date = filter_data.iloc[filter_data['Result'].idxmax()]['Date'].date()
			# print(max_date)
			# print(f"Max in 2015-2022 = {max_level} on {max_date}")
			
			if max_level >= mcl:

				# print(f"**Warning: Max level is at or above MCL**")
				minimum_threshold = max_level * 1.2
				# print(f"New MCL = {minimum_threshold:.2f}")

			else:
				minimum_threshold = mcl

		except Exception as e:
			# print("No max level found")
			# print(e)
			minimum_threshold = mcl
		
		if minimum_threshold == mcl:
			self.minimum_threshold_calculated = False
		else:
			self.minimum_threshold_calculated = True
		
		return minimum_threshold

		
		# test_period = ['2015-01-01','2022-01-01']
		# check_period = ['2022-01-01','2023-01-01']




		
dfs = pd.read_excel(data_path.joinpath("GSA_info.xlsx"), sheet_name=None)
GSAs = [i for i in dfs['public_wells']['GSA'].unique()]


gsa = GSA(dfs,"SKGSA")
ps_code = "CA1010029_022_022"
analyte_title = gsa.chemical_titles[2]


A = Analyte(gsa,ps_code,analyte_title)

print(A.public_name)
# print(A.units)
# print(A.mt)
A.all_df
# A.data.shape[0]



DIBROMOCHLOROPROPANE


,title
0,Cr(VI)\n(mg/L)


In [116]:
class Well:
	def __init__(self,gsa,ps_code):
	
		As = [Analyte(gsa,ps_code,chemical_title) for chemical_title in gsa.chemical_titles]
		# analytes = {A.title:A for A in As}

		blank_As = [A for A in As if A.is_empty]


		# trial 2
		self.table_df = pd.concat([i.all_df for i in As]).pivot_table(
			index=['Date'],
			columns='title',
			values='Result',
			aggfunc='max',
			fill_value="-"
			).sort_values(by='Date',ascending=False)

		for analyte in blank_As:
			self.table_df.insert(
				loc=As.index(analyte),
				column=analyte.title,
				value="-",
			)

		self.mts = [
			A.minimum_threshold for A in As
			]




gsa = GSA(dfs,"SKGSA")
ps_code = "CA1010029_022_022"
W = Well(gsa,ps_code)

# W.table_df.to_clipboard(index=True)
# W.table_df
W.mts


[0.01,
 0.00033600000000000004,
 'NA',
 0.013,
 10.0,
 0.005,
 0.005,
 0.011399999999999999,
 'NA']

In [ ]:


# print(A.public_name)
# print(A.units)
# print(A.mt)
# As.all_df
# pl.concat([i.all_df for i in As])


# trial 1
# table_df = pd.concat([i.all_df for i in As]).reset_index().pivot_table(
# 	index=['index','Date'],
# 	columns='title',
# 	values='Result',
# 	aggfunc='max',
# 	fill_value="-"
# 	).sort_values(by='Date',ascending=False).reset_index().drop(columns=['index'])
# table_df.to_clipboard(index=False)
# table_df



# mts
# As[0].mt

In [ ]:
import xlwings as xw
from pathlib import Path

data = dict(
    title='MyTitle',
    df=pd.DataFrame(data={'one': [1, 2], 'two': [3, 4]})
)

with xw.App(visible=True) as app:
    book = app.render_template(
		'table_template.xlsx',
		'myreport.xlsx',
		**data
		)
    # book.to_pdf(this_dir / 'myreport.pdf')